🎯 1. K-fold Cross Validation là gì?

K-fold Cross Validation (CV) là kỹ thuật đánh giá độ chính xác của mô hình bằng cách chia dữ liệu thành K phần (folds) rồi lặp lại quá trình huấn luyện – kiểm tra K lần.

👉 Mỗi lần:

Lấy (K−1) phần để train mô hình.

Lấy 1 phần còn lại để test.

Sau đó, tính trung bình kết quả Accuracy/F1... của K lần để ra đánh giá cuối cùng.

💡 2. Mục đích của K-Fold CV

| Mục đích                                  | Ý nghĩa                                                                        |
| ----------------------------------------- | ------------------------------------------------------------------------------ |
| 🔁 **Đánh giá mô hình ổn định hơn**       | Giảm ảnh hưởng của việc chia train/test ngẫu nhiên (tránh may rủi).            |
| 📉 **Phát hiện overfitting**              | Nếu accuracy cao trên train nhưng thấp trên nhiều fold → mô hình bị học vẹt.   |
| ⚖️ **So sánh công bằng giữa các mô hình** | Khi so KNN, Decision Tree, SVM... dùng cùng 1 cơ chế chia dữ liệu để đánh giá. |

⚠️ 3. Nếu KHÔNG dùng K-fold Cross Validation thì sao?

Nếu bạn chỉ chia 1 lần train/test (ví dụ 80/20) rồi đánh giá bằng accuracy_score(y_test, y_pred) thì:

✅ Nhanh hơn, ít tốn thời gian tính toán.

❌ Kết quả dễ bị sai lệch — vì phụ thuộc vào cách chia dữ liệu duy nhất.
→ Nếu lần chia đó tình cờ có nhiều mẫu “dễ đoán” ở tập test, accuracy sẽ cao giả tạo.

👉 Nghĩa là mô hình có thể không thật sự tốt, chỉ là gặp may khi chia dữ liệu.

🧠 4. Ví dụ dễ hiểu

Giả sử bạn có 100 mẫu dữ liệu, dùng 5-fold cross validation:

Lần 1: Train trên 80 mẫu (fold 1–4), test trên fold 5.

Lần 2: Train trên fold 1–3,5; test trên fold 4.

…

Lần 5: Train trên fold 2–5; test trên fold 1.

```
Accuracy lần 1: 0.92
Accuracy lần 2: 0.95
Accuracy lần 3: 0.91
Accuracy lần 4: 0.93
Accuracy lần 5: 0.94
Trung bình = 0.93 ± 0.015
```

→ Kết quả ổn định hơn nhiều so với chỉ chia 1 lần.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [3]:
# Đọc dữ liệu
data = pd.read_excel('../../Data/Iris.xls')

# Tách thuộc tính (X) và nhãn (y)
X = data.drop(['iris'], axis=1)
y = data['iris']

# Chia dữ liệu thành train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [4]:
k = int(pow(X_train.shape[0], 1/2) / 2)

models = [
    ('KNN', KNeighborsClassifier(n_neighbors=k)),
    ('DecisionTree', DecisionTreeClassifier(random_state=1)),
    ('SVM', SVC(kernel='rbf', gamma='scale'))
]

In [5]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

print("=== 🔍 ĐÁNH GIÁ MÔ HÌNH BẰNG 10-FOLD CROSS VALIDATION ===\n")
results = {}

for name, model in models:
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=kfold)
    mean_acc = scores.mean()
    std_acc = scores.std()
    results[name] = mean_acc
    print(f"{name:15s} → Accuracy trung bình = {mean_acc*100:.2f}% | Độ lệch chuẩn = {std_acc*100:.2f}%")


=== 🔍 ĐÁNH GIÁ MÔ HÌNH BẰNG 10-FOLD CROSS VALIDATION ===

KNN             → Accuracy trung bình = 96.00% | Độ lệch chuẩn = 4.42%
DecisionTree    → Accuracy trung bình = 95.33% | Độ lệch chuẩn = 4.27%
SVM             → Accuracy trung bình = 96.00% | Độ lệch chuẩn = 3.27%


In [6]:
best_model = max(results, key=results.get)
print(f"\n🏆 Mô hình tốt nhất: {best_model} (Accuracy = {results[best_model]*100:.2f}%)")


🏆 Mô hình tốt nhất: KNN (Accuracy = 96.00%)


5. Giải thích lựa chọn

KNN: Đơn giản, hiệu quả cao với dữ liệu nhỏ, nhưng tốn thời gian khi dự đoán (vì phải tính khoảng cách).

Decision Tree: Dễ hiểu, trực quan, nhưng có thể bị overfitting nếu không giới hạn độ sâu.

SVM: Hoạt động tốt với dữ liệu tuyến tính hoặc có thể tách tuyến tính được, hiệu quả cao nhất trong Iris.